# Advent of code 2024
## Challenge 24
## Part 1
### https://adventofcode.com/2024/day/24#part1

For the first part, I managed to implement everything on my own without help. It was quite alright.

The only thing on which I struggled was to find out that the final answer appeared to be inverted. Then I realized that, as per the problem, the lowest z value was the least significant bit. Meaning the final bit had to be constructed with the z being at the end of the final bit.

I saw in the statistics that a lot of participants only finished the first part of the challenge. In a higher proportion then challenge 21, which was the least completed challenge. So I expected something difficult. And I was not disapointed there.

In [21]:
# The functions performing the different bitwise opeartions on ints.
def AND(a,b):
    if a == 1 and b == 1:
        return 1
    else:
        return 0

def OR(a,b):
    if a == 1 or b == 1:
        return 1
    else:
        return 0

def XOR(a,b):
    if a == 1 and b == 1:
        return 0
    elif a == 0 and b == 0:
        return 0
    else:
        return 1

In [23]:
input_file = open("challenge_24_input.txt", "r")

output_map = {}

queue = []

# In this part, we extract the values of x and y, and immediately
# insert them in the output map. These are wires for which we already know
# the value
for line in input_file:
    
    if line == "\n":
        break
    
    path = line.strip()
    path = path.split(':')
    path[0] = path[0].strip()
    path[1] = path[1].strip()
    path[1] = int(path[1])
    
    output_map[path[0]] = path[1]
 
# for this part, we create a queue, because some 
# do not have values on all their wires when we meet them
# in the queue for the first time, so the queue will allow
# to send send them back in the queue until all their wires
# have a value
for line in input_file:
    line = line.strip()
    line = line.split('->')
    line[0] = line[0].strip()
    line[1] = line[1].strip()
    
    line[0] = line[0].split(' ')
    
    line[0][0] = line[0][0].strip()
    line[0][1] = line[0][1].strip()
    line[0][2] = line[0][2].strip()
    
    # This becomes the final data structure for each gate in the queue
    # to ('qtf', 'XOR', 'nsp', 'ksv')
    queue.append((line[0][0],line[0][1],line[0][2],line[1]))

# We loop through the queue for as long as there are values in it
while queue:

    current_circuit = queue.pop(0)
    
    # If the gate has wires for which there are no values yet in the output map,
    # we send the gate back in the queue
    if current_circuit[0] not in output_map or current_circuit[2] not in output_map:
        queue.append(current_circuit)
        continue
        
    gate_passing = 2
    
    # We perform the gate passing depending on what operations in perform
    if current_circuit[1] == 'AND':
        gate_passing = AND(output_map[current_circuit[0]],output_map[current_circuit[2]])
    elif current_circuit[1] == 'OR':
        gate_passing = OR(output_map[current_circuit[0]],output_map[current_circuit[2]])
    else:
        gate_passing = XOR(output_map[current_circuit[0]],output_map[current_circuit[2]])

    # And we insert the resulting value for the output wire in the output map
    output_map[current_circuit[3]] = gate_passing
 
bit_keys = []

# We extract all the z wires, we put them in a list so that they can
# be sorted fromm smallest to biggest.
for key in output_map:
    if key[0] == 'z':
        bit_keys.append(key)
        
bit_keys.sort()

final_bit = ""

# Here, we insert the value of each wire in a string that will be the final
# we prepend all the new values, so that the lowest z value is the least 
# significant bit.
for key in bit_keys:
    final_bit = str(output_map[key]) + final_bit

# We convert that bit to decimal to get the answer
answer = int(final_bit, 2)

print(answer)

49574189473968


## Part 2

For this part, I did do some exploration of the data, and for example found that there appeared to be one single chain for all z values, but that it would stop earlier to lower z values. So there was a consistency of which wires were used for all z values. 

I found on Reddit a solution based on that finding: 

But in the end I got stuck anyway, and so began looking online and found this solution: https://www.reddit.com/r/adventofcode/comments/1hllrn5/comment/m3nzhx2/

It mentioned that each gate had a set of constraints and that those not following these constraints were the wrong gates.

So I basically implemented the rules that the comment provided, and it worked. This was a bit lame because I essentially got the answer, and just had to implement it in code. But even the implementation required some reflection. It was not that straightforward. 

There was also an exception to the constraints for both gate "x00 XOR y00 -> z00" and for wire z45. The user found those, I don't really know how I could have found those myself. Along with the rules.

I did look at comment that did not provide all the constraints first but then I was stuck for which other rules to find. So then I found the answer.

I also misunderstood the challenge a bit. I thought that we had to *fix* the wrong gates. But no, we only had to find the wrong gates, and list their output wires in ascending order.

In [25]:
input_file = open("challenge_24_input.txt", "r")

and_gates = []
or_gates = []
xor_gates = []
wrong_gates = []

# We ignore the first part, it's not necessary for this
# part of the challenge
for line in input_file:
    if line == "\n":
        break
            
for line in input_file:
    line = line.strip()
    
    # We convert "qtf XOR nsp -> ksv"
    # to [['qtf', 'XOR', 'nsp'], 'ksv']
    # to be able to access each element
    
    line = line.split('->')
    line[0] = line[0].strip()
    line[1] = line[1].strip()
    
    line[0] = line[0].split(' ')
    
    line[0][0] = line[0][0].strip()
    line[0][1] = line[0][1].strip()
    line[0][2] = line[0][2].strip()
    
    # We convert [['qtf', 'XOR', 'nsp'], 'ksv']
    # to ('qtf', 'XOR', 'nsp', 'ksv')
    # to remove the nesting and make it easier to understand
    
    gate = (line[0][0],line[0][1],line[0][2],line[1])
    
    
    if gate[1] == 'XOR':
        # We store the XOR gates in a list because it will be needed for another constraint
        xor_gates.append(gate)
        # The constraint is the following:
        # Either the two input gates are x - y or y - x, or the output wire is a z. If it's not one of those,
        # it's wrong.
        if not (((gate[0][0] == 'x' and gate[2][0] == 'y') or (gate[0][0] == 'y' and gate[2][0] == 'x')) or gate[3][0] == 'z'):
            # I noticed that with those constraints, some gates came up multiple times, so we 
            # check to see if the gate is already in the wrong ones. We only store the output wire
            # because that's what is needed for the answer.
            if gate[3] not in wrong_gates:
                wrong_gates.append(gate[3])
            
    elif gate[1] == 'AND':
        # We store the AND gates in a list because it will be needed for another constraint
        and_gates.append(gate)
        # The rule here is the output wire of an AND gate cannot be z, except for z45.
        # So if the output wire is a z other then z45, it is wrong.
        if gate[3][0] == 'z' and gate[3] != 'z45':
            if gate[3] not in wrong_gates:
                wrong_gates.append(gate[3])
            
    elif gate[1] == 'OR':
        # We store the OR gates in a list because it will be needed for another constraint
        or_gates.append(gate)
        # Here, the constraint is the same as for the AND gates
        if gate[3][0] == 'z' and gate[3] != 'z45':
            if gate[3] not in wrong_gates:
                wrong_gates.append(gate[3])

wrong = True 
# Here, the rule is that the outpute wires of the AND gates have to be an input wire
# for an OR gate. Except if the gate has x00 and y00 as inpute wires.
# So here we ge through all the AND gates.
for and_gate in and_gates:
    if and_gate[0] != 'x00' and and_gate[2] != 'y00':
        # Whe then go through all OR gates to see if the output wire of the AND gate is 
        # part of one of the input wire the of OR gate
        for or_gate in or_gates:
            if and_gate[3] == or_gate[0] or and_gate[3] == or_gate[2]:
                # As soon as we find an input wire in an OR gate matching
                # the output wire of the AND gate, we stop the loop
                wrong = False
                break
        # If wrong is still True, meaning the wire was not 
        # found in any input wire of the OR gates, it is wrong
        if wrong == True:
            # We add the output wire of the gate in the 
            # wrong gates if it is not already there
            if and_gate[3] not in wrong_gates:
                wrong_gates.append(and_gate[3])
        # If there was a positive match, we reset the value
        # to true for the next iteration
        elif wrong == False:
            wrong = True

# Here, this constraint is that the output wire of a XOR gate
# has to also be an inpute wire for another XOR gate.
# The logic is the same as for the constraint above
wrong = True
for xor_gate in xor_gates:
    # One exception is the "x00 XOR y00 -> z00" gate.
    if xor_gate[0] != 'x00' and xor_gate[2] != 'y00' and xor_gate[3] != 'z00':
        if (xor_gate[0][0] == 'x' or xor_gate[0][0] == 'y') and (xor_gate[2][0] == 'x' or xor_gate[2][0] == 'y'):
            for second_xor_gate in xor_gates:
                if xor_gate[3] == second_xor_gate[0] or xor_gate[3] == second_xor_gate[2]:
                    wrong = False
            if wrong == True:
                if xor_gate[3] not in wrong_gates:
                    wrong_gates.append(xor_gate[3])
            elif wrong == False:
                wrong = True

# We sort the list of the output wires of the wrong gates
wrong_gates.sort()

# we join them with commas and we have the answer
print(",".join(wrong_gates))

ckb,kbs,ksv,nbd,tqq,z06,z20,z39
